In [186]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook

In [119]:
xl_marketdata = pd.ExcelFile('db_marketdata.xlsx')

In [165]:
asset = pd.read_excel(xl_marketdata, sheet_name='price', header=[8], index_col=[0]).head(1).T
asset.columns.name = None
asset.columns = ['name']
# asset['id'] = asset.index.map(lambda x: int(x[1:]))
asset

,name
A005930,삼성전자
A000660,SK하이닉스
A005380,현대차
A068270,셀트리온
A051910,LG화학
A207940,삼성바이오로직스
A012330,현대모비스
A005490,POSCO
A051900,LG생활건강
A055550,신한지주


In [122]:
price = pd.read_excel(xl_marketdata, sheet_name='price', header=[8], index_col=[0]).iloc[5:]
price.index.name = None
# price

In [124]:
nshares = pd.read_excel(xl_marketdata, sheet_name='nshares', header=[8], index_col=[0]).iloc[5:]
nshares.index.name = None
# nshares

In [126]:
tradable = pd.read_excel(xl_marketdata, sheet_name='tradable', header=[8], index_col=[0]).iloc[5:]
tradable.index.name = None
# tradable

In [128]:
amount = pd.read_excel(xl_marketdata, sheet_name='amount', header=[8], index_col=[0]).iloc[5:]
amount.index.name = None
# amount

In [192]:
def get_asset(ticker):
    return {
        'model': 'api.asset', 
        'pk': int(ticker[1:]), 
        'fields': {
            'ticker': ticker, 
            'name': asset.loc[ticker, 'name']
        }
    }
    
def get_marketdata(ticker):
    _marketdata = []
    
    for date in price.index:
        _price = price.loc[date, ticker]
        _nshares = nshares.loc[date, ticker]
        _amount = amount.loc[date, ticker]
        
        if (_price is not np.nan) and (_nshares is not np.nan) and (_amount is not np.nan):        
            _marketdata.append({
                'model': 'api.marketdata', 
                'pk': int(''.join(str(date.date()).split('-')) + ticker[1:]), 
                'fields': {
                    'asset': int(ticker[1:]), 
                    'date': str(date.date()), 
                    'price': float(_price), 
                    'nshares': int(_nshares), 
                    'tradable': True if tradable.loc[date, ticker]=='정상' else False, 
                    'amount': int(_amount)
                }
            })
            
    return _marketdata

In [207]:
dbinit = []
for ticker in tqdm_notebook(asset.index):
    dbinit.append(get_asset(ticker))
    dbinit.extend(get_marketdata(ticker))

In [208]:
with open('dbinit.json', 'w') as f:
    json.dump(dbinit, f)

In [194]:
get_marketdata('A207940')

[{'model': 'api.marketdata',
  'pk': 20161130207940,
  'fields': {'asset': 207940,
   'date': '2016-11-30',
   'price': 154000.0,
   'nshares': 66165000,
   'tradable': True,
   'amount': 337566572167}},
 {'model': 'api.marketdata',
  'pk': 20161231207940,
  'fields': {'asset': 207940,
   'date': '2016-12-31',
   'price': 151000.0,
   'nshares': 66165000,
   'tradable': True,
   'amount': 38863340040}},
 {'model': 'api.marketdata',
  'pk': 20170131207940,
  'fields': {'asset': 207940,
   'date': '2017-01-31',
   'price': 161000.0,
   'nshares': 66165000,
   'tradable': True,
   'amount': 25085911839}},
 {'model': 'api.marketdata',
  'pk': 20170228207940,
  'fields': {'asset': 207940,
   'date': '2017-02-28',
   'price': 164500.0,
   'nshares': 66165000,
   'tradable': True,
   'amount': 36988665251}},
 {'model': 'api.marketdata',
  'pk': 20170331207940,
  'fields': {'asset': 207940,
   'date': '2017-03-31',
   'price': 173000.0,
   'nshares': 66165000,
   'tradable': True,
   'amount':

In [202]:
asset.index

Index(['A005930', 'A000660', 'A005380', 'A068270', 'A051910', 'A207940',
       'A012330', 'A005490', 'A051900', 'A055550',
       ...
       'A900060', 'A900090', 'A900130', 'A900150', 'A900180', 'A950010',
       'A950030', 'A950070', 'A950100', 'A950180'],
      dtype='object', length=3087)

In [198]:
test = []
for ele in dbinit:
    if ele['model']=='api.asset':
        test.append(ele['fields']['name'])

In [199]:
len(test)

3087

In [206]:
[1,3,3,9].count(3)

2